<h1 align=center><font size = 5>JT's IBM Data Science Capstone Assignment:</font></h1>

<h1 align=center><font size = 5>Red and Blue Gerrymandering</font></h1>

<h3 align=center><font size = 3>Responding to (alleged) unfair Georgia redistricting  </font></h3>

<a><img align=center src = https://www.economicmodeling.com/wp-content/uploads/2017/08/georgia-peach.jpg width = 200> </a>

_(image courtesy of economicmodeling.com)_

<h3 align=left><font size = 4>Introduction (Business Problem Summary):</font></h3>

Amongst the issues affecting election campaigning, Gerrymandering is one of the most vexing and arcane.  Given the contentious nature of this topic in Georgia (USA) and the close results of recent elections, it is important that political campaign management firms (as well as voters) understand alternatives to counter effects of this issue in order to better target candidate marketing strategies during the next Georgia elections. 

Gerrymandering is a practice intended to establish an unfair political advantage for a particular party or group by manipulating US congressional election district boundaries. "Districts" define geographical boundaries, with each district within a state being geographically contiguous and having about the same number of state voters.  The redistricting process is managed by committee, currently a committee chaired by the Republican party and is expected to happen following the 2020 US census.  The last redistricting in Georgia occurred in 2015 and had come under fire due to suspected gerrymandering to suppress minority votes (source "https://democraticredistricting.com/state/georgia/").  This report explores the demographics for the districts and those of counties across the state, *ignoring district boundaries*, in order to better understand make-up and potential avenues for combatting gerrymandering effects.

**The target audience for this analysis are political campaign management firms who do business in Georgia.**  Such firms typically control campaign investment strategies, including types of adverts, locations to focus on, and delivery channels for candidate communication.  Such strategies are aimed at achieving the highest voter turnout for the candidate, **therefore, demographics analysis is vital to the success and reputation of the campaign management firm** (as well as the candidate).


_Caveats: this is exploratory analysis (in the loosest sense); results and observations are limited to datasets available to students and in the context of the IBM Data Science Professional course schedule.  Moreover, this report does not express a political preference.  At a time where complete and accurate information ("truth") is under stress, it is important that readers bear these points in mind re: this notebook and report._

_Also be aware, this analysis required quite a few disparate files and so is somewhat longer than notebooks referenced in course samples_

<h3 align=left><font size = 3>Data Requirements:</font></h3>

Certain characteristics of the 14 "red" (Republican) and  "blue" (Democratic) districts will be reviewed:
- voter population size
- ethnicity (using 2015 survey estimates related by BallotPedia)
- education levels of voters
- age ranges of voters
- poverty levels in each district
- local amenities (venue categories) for each district


The features will be sourced from following:

*** Election results too old but used the ethnicity estimates as could not find anything better***
https://ballotpedia.org/Redistricting_in_Georgia - will provide congressional districts by number, current representative by full name, party affiliation, election victory margins, and district ethnic demographic estimates (BallotPedia referenced the United States Census Bureau, "American Fact Finder: 2015 1-year estimates," for ethnicity data).

https://en.wikipedia.org/wiki/2018_United_States_House_of_Representatives_elections_in_Georgia - will provide congressional district 2018 election results for Georgia.

https://www2.census.gov/programs-surveys/demo/tables/voting/table02c.xlsx - "Characteristics (Educational Attainment) of the Citizen Voting-Age Population for Congressional Districts: 2018" - will provide education attainment per district.

https://www2.census.gov/programs-surveys/demo/tables/voting/table02a.xlsx - "Characteristics (Age) of the Citizen Voting-Age Population for Congressional Districts: 2018" - will provide voter age per district.

https://www2.census.gov/programs-surveys/demo/tables/voting/table02b.xlsx - "Characteristics (Sex and Poverty) of the Citizen Voting-Age Population for Congressional Districts: 2018" - will provide number of people living in poverty for each district.

"./Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/Coursera_Capstone/georgia_districts_addr" - manually created from the .gov websites of each congressional representative (as per BallotPedia Georgia Districts html table), using the first office address where representatives may have had multiple addresses.

https://developer.foursquare.com/docs/build-with-foursquare/categories - FourSquare will be used to select venue information for each district. Such data will further inform the philosophy and voting pattern potential for a given district.

https://www.census.gov/geographies/mapping-files/2018/geo/kml-cartographic-boundary-files.html ("cb_2018_13_sldu_500k") map of Georgia USA with county boundaries, converted to geojson using the open-source tool kml2geojson.

Below data sources are county-level detail that will be used in the experimental reclustering of counties based on demographics and not congressional district: 


_**(Important note - counties are NOT subsets of districts)**_

"./Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/Coursera_Capstone/CAINC30_GA_1969_2018.csv" - manually extracted from CAINC30 zip file downloaded from the "US Dept of Commerce - BEA Regional Economics Data" (https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=1&isuri=1&acrdn=8#reqid=70&step=1&isuri=1&acrdn=8 ).  Will provide population and income by county ('LineCodes' 100 and 110).

http://sos.ga.gov/admin/files/Active_Voters_by_Race_Gender_and_Age_as_of_May_1_2020.xlsx - from Georgia Voter Registration Statistics goverment site - active voter numbers by race and gender.


<h3 align=left><font size = 3>Methodology:</font></h3>

In order to suggest meaningful focus areas for campaign advisors, several steps will be followed:

1. **Data sourcing for Districts** - Useful data sources on Georgia congressional districts will be identified with information that is as current as possible (see "Data" section above). The data will then be loaded into a common 'merged' table which can be easily charted and evaluated (with '.head()' checks to insure data preprocessing efficacy).  Note that data will mostly be sourced from public and government websites with the exception of:
    + district center-point addresses, which are more difficult to source and required a manually-derived table
    + amenities information for districts of interest; this will be provided via FourSquare apis.


2. **Data visualisation and Analysis for Districts** - Data visualisations will be created that illustrate demographic breakdowns for the districts.  Highlights and anomalies will be noted, with a view to narrowing focus to a few districts with the most interesting characteristics.  Further elaboration data will be charted for the shortlist of high-interest districts to understand types of amenities available in each. Additionally, a broad clustering of counties across Georgia will be performed in order to identify potential counties of interest for alternative campaign targetting.


3. **Data Sourcing, Visualisation and Analysis for Counties** - Assuming the previous District-focussed processes have yielded interesting information, repeat the Data sourcing and visualisation steps above for Counties (smaller geographical representations) in Georgia to identify clusters of counties that might further inform better ways of targetting voters than reliance on district boundaries.


4. **Results discussion** - Consolidate observations from above steps, contrasting District information.  Assess results, unusual observations, and other process observations that may be useful.


5. **Conclusions** - The final recommendations for the campaign managers, advisors and consultants.


<h3 align=left><font size = 3>Step 1 - Data sourcing:</font></h3>

First, grab necessary libraries:

In [ ]:
#get basic dataframe libraries
import pandas as pd
import numpy as np

#get libraries to help url gets (for Beautiful Soup, FourSquare, etc)
import json
import requests
from pandas.io.json import json_normalize

#get mapping & charting libraries
%matplotlib inline 
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

plt.style.use('ggplot')
mpl.style.use('ggplot') 
import folium
from folium import plugins
from folium.map import Layer

#import to control fig rendering space (helps with desktop and html)
from branca.element import Figure

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

!conda install -c conda-forge geocoder --yes
import geocoder 

from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup

#to get rid of special characters from web pages, if necessary
import string
ValidChar = string.ascii_letters+string.punctuation

#I'm using my Macbook Anaconda Jupyter environment so need wget
import wget


Create a dataframe of districts (of which there are 14) and associated data from the sources listed in the Data section above.  

Seed the dataframe using one of the Ballotpedia tables (there are a few tables in the page; use the "Electoral Margins of Victory...") via BeautifulSoup:

In [ ]:
url="https://en.wikipedia.org/wiki/2018_United_States_House_of_Representatives_elections_in_Georgia"
req = requests.get(url)
soup = BeautifulSoup(req.content,'lxml')


In [ ]:
#grab the 14th table format in the webpage and convert to a dataframe
table = soup.find_all('table')[13]

table_list = pd.read_html(str(table))
df_districts=pd.DataFrame(table_list[0])

#df_districts.head()

Clean this dataframe up a bit:
- replace column labels, 
- drop "Top opponent"s, "Georgia" and "%",  
- convert VictoryMargin to float % and 
- convert VotesCast to int
- add text reflecting political party of district

In [ ]:
df_districts.columns = ['District', 'Republican Votes', 'Republican%',
                        'Democratic Votes', 'Democratic%', 
                        'Other Votes','Other%', 'Total Votes', 
                        'Total%', 'Election summary'] 

todrop = ['Republican%', 'Democratic%', 'Other Votes', 
          'Other%', 'Total Votes', 'Total%']
df_districts.drop(todrop, axis=1, inplace=True)
df_districts.drop([14], axis=0, inplace=True)


In [ ]:
district_cols = df_districts.columns

df_districts['District'].str.strip(ValidChar).astype(bool).any()
df_districts['Election summary'].str.strip(ValidChar).astype(bool).any()

df_districts['District'] = \
  df_districts['District'].str.replace("District ","")

df_districts['District'] = \
  df_districts['District'].astype(str).astype(int)

df_districts[district_cols[1:2]] = \
  df_districts[district_cols[1:2]].astype(str).astype(int)

#add column to describe voting result margin, Republican - Democrat
df_districts['Rep-Dem Margin'] = \
  df_districts['Republican Votes'] - df_districts['Democratic Votes']


In [ ]:
df_districts['Party']=None

for i, margin in enumerate(df_districts['Rep-Dem Margin']):
    if margin >0:
        df_districts.iloc[i, df_districts.columns.get_loc('Party')] = \
          "Republican"
    elif margin <0:
        df_districts.iloc[i, df_districts.columns.get_loc('Party')] = \
          "Democratic"
    else:
        df_districts.iloc[i, df_districts.columns.get_loc('Party')] = \
          "Error"    
    
print('Shape: ', df_districts.shape)
df_districts.head(3)

Add the voting data by ethnicity, following the same process as done for the dataframe seeding above:

In [ ]:
#grab the 11th table format in the BallotPedia webpage and convert 
#to a dataframe
url2="https://ballotpedia.org/Redistricting_in_Georgia"

req = requests.get(url2)
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[10]

table_list = pd.read_html(str(table))
df_demogs = pd.DataFrame(table_list[0])

#df_demogs.head(3)


...and as with previous load, clean this table:

In [ ]:
#remove header from column labels by renaming columns
demog_columns = ['Distr', 'Hispanic or Latino of any race', \
                    'White', 'Black or African American', \
                    'American Indian and Alaska Native', 'Asian', \
                    'Native Hawaiian and other Pacific Islander', \
                    'Other', 'Multiple races', 'todrop0']
df_demogs.columns = demog_columns


In [ ]:
#get rid of superfluous or special characters
df_demogs['Distr'].str.strip(ValidChar).astype(bool).any()

df_demogs['Distr'] = \
  df_demogs['Distr'].str.replace(", Georgia","")
df_demogs['Distr'] = \
  df_demogs['Distr'].str.replace("District","")

#get rid of superfluous row and column
df_demogs.drop([14], axis=0, inplace=True)
df_demogs.drop(['todrop0'], axis=1, inplace=True)


In [ ]:
#convert values to int (for later)
demog_columns = df_demogs.columns

df_demogs[demog_columns[1:]] = \
  df_demogs[demog_columns[1:]].astype(str).astype(int)

df_demogs['Distr'] = df_demogs['Distr'].astype(str).astype(int)

df_demogs.head(3)


Merge the demographics df into the main df (df_districts_merged):

In [ ]:
df_districts_merged = pd.merge(left=df_districts, right=df_demogs, 
                        left_on='District', right_on='Distr')

df_districts_merged.drop(['Distr'], axis=1, inplace=True)

print('shape: ', df_districts_merged.shape)
df_districts_merged.head(3)


Add the district data on education, age, and poverty for districts in Georgia ("GA") in the cells below, cleansing as appropriate:

In [ ]:
#pull education data
url_ed = \
 "https://www2.census.gov/programs-surveys/demo/tables/voting/table02c.xlsx"

df_education = pd.read_excel(url_ed, skiprows=range(3))

column_names = df_education.columns.values
column_names[1] = "State"
column_names[3] = "Districttodrop"
df_education.columns = column_names


In [ ]:
tmp_df = df_education[['State', 'Districttodrop','Less than 9th grade', \
                     'High school or more', \
                      "Bachelor's degree or more"]]

df_ed = tmp_df[tmp_df['State'] == "GA"]


In [ ]:
df_districts_merged = pd.merge(left=df_districts_merged, right=df_ed, 
                        left_on='District', right_on='Districttodrop')

df_districts_merged.drop(['Districttodrop', 'State'], axis=1, inplace=True)

print('shape: ', df_districts_merged.shape)
df_districts_merged.head(3)

In [ ]:
#pull age data
url_age = \
  "https://www2.census.gov/programs-surveys/demo/tables/voting/table02a.xlsx"

df_age = pd.read_excel(url_age, skiprows=range(3))
#df_age.head()

In [ ]:
cols = df_age.columns.values

cols[1] = "State"
cols[3] = "Districttodrop"
cols[4] = "Voting Pop"
df_age.columns = cols


In [ ]:
tmp_df = df_age[['State', 'Districttodrop', 'Voting Pop', \
                '18-29 years old', '30-44 years old',
                '45-64 years old', \
                '65 years old and older']]

df_age = tmp_df[tmp_df['State'] == "GA"]


In [ ]:
df_districts_merged = pd.merge(left=df_districts_merged, right=df_age, 
                        left_on='District', right_on='Districttodrop')

df_districts_merged.drop(['Districttodrop', 'State'], axis=1, inplace=True)

print('shape: ', df_districts_merged.shape)
df_districts_merged.head(3)

In [ ]:
#pull poverty data
url_poverty = \
"https://www2.census.gov/programs-surveys/demo/tables/voting/table02b.xlsx"

df_poverty = pd.read_excel(url_poverty, skiprows=range(3))


In [ ]:
cols = df_poverty.columns.values

cols[1] = "State"
cols[3] = "Districttodrop"
df_poverty.columns = cols


In [ ]:
tmp_df = df_poverty[['State', 'Districttodrop','Below poverty level']]

df_poverty = tmp_df[tmp_df['State'] == "GA"]


In [ ]:
df_districts_merged = pd.merge(left=df_districts_merged, 
                               right=df_poverty, 
                        left_on='District', right_on='Districttodrop')

df_districts_merged.drop(['Districttodrop', 'State'], axis=1, 
                         inplace=True)

print('shape: ', df_districts_merged.shape)
df_districts_merged.head(3)

In [ ]:
#Divide numbers by 1000 and create merged label for plotting charts 
#later on
cols = df_districts_merged.columns.values

df_districts_merged[cols[6:]] = df_districts_merged[cols[6:]]/1000

df_districts_merged['District - Party'] = \
    df_districts_merged['District'].astype(str) + ' - ' + \
    df_districts_merged['Party']

print('shape: ', df_districts_merged.shape)
df_districts_merged.head(3)

Add geolocations for each district (using geocoder) for later mapping and venues per district contrasting.  As there is no convenient source for district centerpoint addresses, district address information was sourced from individual congressional representative websites (one address per district, using first address in "Office Location" pages)

In [ ]:
csv_path="/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/Coursera_Capstone/georgia_distr_addr.csv"
temp_df=pd.read_csv(csv_path)


In [ ]:
df = temp_df
cols = df.columns.values
cols[0] = "Distr"
cols[1] = "rep"
df.columns = cols
df['Latitude']=None
df['Longitude']=None


In [ ]:
#Load the longitudes and latitudes for the various district offices

for i, office in enumerate(df['DistrictOffice']):
    g = geocoder.arcgis(office)
    df.iloc[i, df.columns.get_loc('Latitude')] = g.latlng[0]
    df.iloc[i, df.columns.get_loc('Longitude')] = g.latlng[1]

df.head(3)

In [ ]:
df_districts_merged = \
  pd.merge(left=df_districts_merged, right=df, 
                        left_on='District', right_on='Distr')

df_districts_merged.drop(['Distr', 'rep', 'DistrictOfficeAddr'], 
                         axis=1, inplace=True)

print('shape: ', df_districts_merged.shape)
df_districts_merged.head(3)

In [ ]:
#set associated party colors (handy for visualisations later) 

df_districts_merged['party_color']=""

df_districts_merged.loc[(df_districts_merged['Party'] == 
            "Republican"), 'party_color'] = "indianred"

df_districts_merged.loc[(df_districts_merged['Party'] == 
            "Democratic"), 'party_color'] = "cornflowerblue"

In [ ]:
df_districts_merged.head(3)

***********************************************************************

<h3 align=left><font size = 3>Step 2 - Data visualisation and Analysis:</font></h3>

***Chart observations to evaluate districts that may be of most interest to campaign advisors and consultants.***


In [ ]:
#show ethnicity across districts.  Consolidate smaller columns
#into the 'Other' category strictly for bar chart formatting (not
#because the numbers aren't important)
cols=['District - Party', 'Hispanic or Latino of any race', 
     'White', 'Black or African American',
      'American Indian and Alaska Native',
     'Asian', 'Native Hawaiian and other Pacific Islander',
     'Other', 'Multiple races']

#df_ethnicity = pd.DataFrame(columns=cols)
df_ethnicity = pd.DataFrame(data=None)
df_ethnicity[cols] = df_districts_merged[cols]

df_ethnicity['Other'] = df_ethnicity['Other'] + \
    df_ethnicity['American Indian and Alaska Native'] + \
    df_ethnicity['Native Hawaiian and other Pacific Islander'] + \
    df_ethnicity['Multiple races']

df_ethnicity.drop(['American Indian and Alaska Native',
                  'Native Hawaiian and other Pacific Islander',
                  'Multiple races'], axis=1, inplace=True)

#and add % for later df display
cols = df_ethnicity.columns.values
ethnicity_tot = df_ethnicity[cols[1:]].sum(axis=1)

df_ethnicity['Hispanic/ Latino %'] = \
    ( (df_ethnicity['Hispanic or Latino of any race'] / \
     ethnicity_tot) * 100).round(2)
df_ethnicity['White %'] = ( (df_ethnicity['White'] / \
     ethnicity_tot) * 100).round(2)
df_ethnicity['Black/ African American %'] = \
    ( (df_ethnicity['Black or African American'] / \
     ethnicity_tot) * 100).round(2)
df_ethnicity['Asian %'] = ( (df_ethnicity['Asian'] / \
     ethnicity_tot) * 100).round(2)
df_ethnicity['Other %'] = ( (df_ethnicity['Other'] / \
     ethnicity_tot) * 100).round(2)


In [ ]:
cols=['District - Party', 'Hispanic or Latino of any race', 
     'White', 'Black or African American',
     'Asian', 'Other']

colorlist=['coral', 'palevioletred', 
           'forestgreen', 'skyblue', 'darkseagreen'] 

labels = df_ethnicity['District - Party']

x=np.arange(0,600,100)
ax=df_ethnicity[cols].plot(kind='bar', 
                   figsize=(16,10), 
                   color=(colorlist))

ax.set_title("Ethnicity by District (2015 Estimate)",
             fontsize=12)

ax.set_yticks(x)

ax.set_xticklabels(labels, fontsize=10, rotation=70)


ax.set_xlabel("District / Party", fontsize=10)
ax.set_ylabel("In 000's", fontsize=10)

plt.legend(fontsize=10)

plt.figure(figsize=(16,10))

plt.show()

In [ ]:
#show top 3 districts by 'Hispanic or Latino of any race' voting 
#population size
df_display = \
  df_ethnicity[['District - Party', 
              'Hispanic or Latino of any race', 
              'Hispanic/ Latino %']].sort_values(by =
                            ('Hispanic or Latino of any race'), \
                         ascending=False).head(3)
df_display.style.hide_index()

In [ ]:
#show top 3 districts by 'White' voting population size
df_display = df_ethnicity[['District - Party',
                           'White', 'White %']].sort_values(by = 
                            ('White'), ascending=False).head(3)
df_display.style.hide_index()

In [ ]:
#show top 3 districts by 'Black or African American' voting population 
#size
df_display = df_ethnicity[['District - Party',
              'Black or African American', 
             'Black/ African American %']].sort_values(by = 
                         ('Black or African American'), \
                         ascending=False).head(3)
df_display.style.hide_index()

In [ ]:
#show top 3 districts by 'Asian' and 'Other' voting population sizes
df_display = df_ethnicity[['District - Party','Asian', 
                           'Other', 'Asian %', 
                          'Other %']].sort_values(by = 
                         (['Asian', 'Other']), \
                         ascending=False).head(3)
df_display.style.hide_index()

Data shows that, broadly speaking, districts with a higher number of white constituents relative to the rest of the district population tend to have elected a Republican candidate and districts with a higher number of black constituents tend to have elected a Democratic candidate.  The exception is District 6, where the number of white constituents relative to the rest of the district population is greatest.  

In [ ]:
#show education across districts
cols=['District - Party', 'Less than 9th grade', 
      'High school or more',
       "Bachelor's degree or more"]

colorlist=['lightcoral', 'mediumseagreen', 'orchid']

labels = df_districts_merged['District - Party']

x=np.arange(0,600,100)
ax=df_districts_merged[cols].plot(kind='bar', 
                   figsize=(16,10), 
                   color=(colorlist))

ax.set_title("Education by District (2018 census data)",
             fontsize=12)

ax.set_yticks(x)

ax.set_xticklabels(labels, fontsize=12, rotation=70)


ax.set_xlabel("District / Party", fontsize=10)
ax.set_ylabel("In 000's", fontsize=10)

plt.legend(fontsize=10)

plt.figure(figsize=(16,10))

plt.show()

In [ ]:
df_display = pd.DataFrame(data=None)

In [ ]:
#show top 3 districts with highest education attainment

df_display = df_districts_merged[cols].sort_values(by = 
                    ("Bachelor's degree or more"), \
                                      ascending = False)
df_display = df_display[['District - Party', 
                         "Bachelor's degree or more"]].head(3)
df_display.style.hide_index()

In [ ]:
#show the 3 districts with least education (highest below 9th grade)
df_display = pd.DataFrame(data=None)
df_display = df_districts_merged[cols].sort_values(by = 
                    ('Less than 9th grade'), \
                                      ascending = False)
df_display = df_display.head(3)
df_display[['District - Party',
            'Less than 9th grade']].style.hide_index()

Data shows that District 6 has the highest number of college graduates, whilst District 9 has the highest number of people who did not enter highschool.  

In [ ]:
#show voting age demographics across districts
cols=['District - Party', '18-29 years old', '30-44 years old', 
       '45-64 years old', '65 years old and older']

colorlist=['lightcoral', 'mediumseagreen', 'orchid', 'deepskyblue']

labels = df_districts_merged['District - Party'] 

x=np.arange(0,350,50)
ax=df_districts_merged[cols].plot(kind='bar', 
                   figsize=(16,10), 
                   color=(colorlist))

ax.set_title("Age by District (2018 census data)",
             fontsize=12)

ax.set_yticks(x)

ax.set_xticklabels(labels, fontsize=10, rotation=70)


ax.set_xlabel("District / Party", fontsize=10)
ax.set_ylabel("In 000's", fontsize=10)

plt.legend(fontsize=10)

plt.figure(figsize=(16,10))

plt.show()


In [ ]:
#add the relative voting by age
df_rel_voting = pd.DataFrame(data=None) 

df_rel_voting['District - Party'] = \
    df_districts_merged['District - Party']

df_rel_voting['18-29 years old %'] = \
    ((df_districts_merged['18-29 years old'] / \
      df_districts_merged['Voting Pop']) * 100).astype(float).round(2)

df_rel_voting['30-44 years old %'] = \
    ((df_districts_merged['30-44 years old'] / \
      df_districts_merged['Voting Pop']) * 100).astype(float).round(2)

df_rel_voting['45-64 years old %'] = \
    ((df_districts_merged['45-64 years old'] / \
      df_districts_merged['Voting Pop']) * 100).astype(float).round(2)

df_rel_voting['65 years old and older %'] = \
    ((df_districts_merged['65 years old and older'] / \
      df_districts_merged['Voting Pop']) * 100).astype(float).round(2)

df_rel_voting['Total Voting-Aged Pop (000s)'] = \
    df_districts_merged['Voting Pop']


In [ ]:
#show top 3 districts for 18-29 year old voting population %
df_display = df_rel_voting[['District - Party',
                            '18-29 years old %']].sort_values(by=
                    ('18-29 years old %'), ascending=False).head(3)
df_display.style.hide_index()

In [ ]:
#show top 3 districts for 30-44 year old voting population %
df_display = df_rel_voting[['District - Party',
                            '30-44 years old %']].sort_values(by=
                    ('30-44 years old %'), ascending=False).head(3)
df_display.style.hide_index()

In [ ]:
#show top 3 districts for 45-64 year old voting population %
df_display = df_rel_voting[['District - Party',
                            '45-64 years old %']].sort_values(by=
                    ('45-64 years old %'), ascending=False).head(3)
df_display.style.hide_index()

In [ ]:
#show top 3 districts for 45-64 year old voting population %
df_display = df_rel_voting[['District - Party',
                        '65 years old and older %']].sort_values(by=
                ('65 years old and older %'), ascending=False).head(3)
df_display.style.hide_index()

Age data shows District 5 to have the largest voting participation from the voting-aged constituents in the 18-44 years old range whilst District 6 appears to have the greatest participation from 45-64 year old constituents and District 9 has the greatest participation from constituents 65 years and older.  

In [ ]:
#show relative poverty across districts
cols=['District - Party', 'Voting Pop', 'Below poverty level']

colorlist=['lightcoral', 'mediumseagreen']

labels = df_districts_merged['District - Party']

#plot population and poverty next to each other using 'subplot' (not taught
#in the course, but quite handy and easy to learn from matplotlib doc)
fig, ax=plt.subplots(ncols=2, figsize=(20,8))

ytcks=np.arange(0,850,100)
ax1=df_districts_merged['Voting Pop'].plot(kind='bar',
                    title="Voting Population (000s)",
                   color=(colorlist[0]), ax=ax[0])
#add totals at top of bar for better visualisation
for i in ax1.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax1.annotate('{0:.1f}'.format(height), (x + -.2, y + height + 6), fontsize=10, 
                 color='black')
ax1.set_yticks(ytcks)
ax1.set_xticklabels(labels, fontsize=10, rotation=70)


#plot a bar chart for each cluster's stat result
ytcks2=np.arange(0,150,25)
ax2=df_districts_merged['Below poverty level'].plot(kind='bar', 
    title='People living below poverty level (000s)',
    fontsize=12, rot=0, color='cadetblue', ax=ax[1])
#add totals at top of bar for better visualisation
for i in ax2.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax2.annotate('{:}'.format(height), (x - 0.2, y + height + 1.0), fontsize=10, 
                 color='black')
ax2.set_yticks(ytcks2)
ax2.set_xticklabels(labels, fontsize=10, rotation=70)

plt.figure(figsize=(16,10))

plt.show()

#show the 3 districts with highest poverty levels

df_display = \
    df_districts_merged[cols].sort_values(by=('Below poverty level'), \
                                     ascending = False)
df_display = df_display.head(3)
df_display.rename({'Voting Pop':'Voting Pop (000s)'}, axis=1, 
                  inplace=True)
df_display.rename({'Below poverty level':'Below poverty level (000s)'}, 
                  axis=1, inplace=True)
df_display.style.hide_index()

#show the 3 districts with lowest poverty levels, lowest first
df_display = \
  df_districts_merged[cols].sort_values(by=('Below poverty level'), \
                                     ascending = True)
df_display.rename({'Voting Pop':'Voting Pop (000s)'}, axis=1, 
                  inplace=True)
df_display.rename({'Below poverty level':'Below poverty level (000s)'},
                  axis=1, inplace=True)
df_display=df_display.head(3)
df_display.style.hide_index()

Poverty data shows highest levels of poverty in Districts 2, 8, 5, with Districts 10 & 12 in close proximity to District 5.  Districts 2 and 5 are Democratic and Districts 8, 10 and 12 are Republican, suggesting that poverty level is not necessarily a party vote differentiator.  Note that District 6 appears to have the lowest level of poverty in Georgia.  

In [ ]:
#show the election victory margins for each district, to help focus
#analysis
df_display=pd.DataFrame(data=None)
df_display = df_districts_merged[['District - Party', 
                                  'Election summary', 
                                  'Republican Votes',
                                'Democratic Votes', 
                                  'Rep-Dem Margin']].copy()

df_display.style.hide_index()

Based on data visualised and analysis thus far, interesting districts for campaign advisor priority review are:
- District 6 - unusual race demographic relative to political party elected, highest number of voters aged between 45-64 years old, highest educational attainment and lowest poverty level.  It is also the only district to have elected a different party in 2018, albeit by the second narrowest margin.
- District 2 - highest poverty level, lowest educational attainment
- District 9 - highest population of white voters, lowest educational attainment, greatest percentage of older voters (65 and older) and largest election victory margin (discounting unopposed elections in Districts 5 & 8).

Now pull amenities data (using the FourSquare service) for all Georgia
congressional districts to see other potentially defining characteristics across districts.  Results will be formatted into a 'top venues' chart to illustrate district similarities/differences and potential ad campaign targets.

In [ ]:
#create new df for the venue information
df_district_venuetypes = \
  df_districts_merged[['District', 'DistrictOffice', 'Party', 
                       'Voting Pop', 
                      'Latitude', 'Longitude']]

print('shape: ', df_district_venuetypes.shape)
df_district_venuetypes.head(3)

In [ ]:
#load credentials
filename1="/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/fsq.clientid"
with open(filename1) as file1:
    CLIENT_ID=file1.read()

filename2="/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/fsq.clientsec"
with open(filename2) as file2:
    CLIENT_SECRET=file2.read()
    
#and limit the FourSquare returned data to up to 1000 venues located 
#within 3200 meters (approx 2 miles) of each district's office.
RADIUS=3200
LIMIT=1000
VERSION = '20180605'

In [ ]:
#convenient function to construct a venues df from a loop
def getNearbyVenues(district, names, party, population, 
                    latitudes, longitudes, radius=500):    
    venues_list=[]
    for distr, name, pty, pop, lat, lng in zip(district, names, 
                                               party, population, 
                                               latitudes, longitudes):
        print("district ", distr, " ", name)            
        url = \
        'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, 
            LIMIT)            
        results = \
            requests.get(url).json()["response"]['groups'][0]['items']   
        venues_list.append([(distr, name, pty, pop, lat, lng,  
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results] )

    nearby_venues = pd.DataFrame([item for venue_list in 
                                  venues_list for item in venue_list])
    nearby_venues.columns = ['District', 'Location', 'Party', 
                            'Voting Pop', 
                            'Location Latitude', 
                            'Location Longitude', 
                            'Venue', 
                            'Venue Latitude', 
                            'Venue Longitude', 
                            'Venue Category']
    
    return(nearby_venues)

In [ ]:
#get district venues using function defined earlier (uses 
#FourSquare)
district_venues = \
  getNearbyVenues(district=df_district_venuetypes['District'],
                  names=df_district_venuetypes['DistrictOffice'],
                  party=df_district_venuetypes['Party'],
                  population=df_district_venuetypes['Voting Pop'],
                latitudes=df_district_venuetypes['Latitude'],
                longitudes=df_district_venuetypes['Longitude'],
                 radius=RADIUS)

In [ ]:
#set a reasonable limit for the number of district preferences to 
#review (this may seem somewhat arbitrary but we have to start 
#somewhere...also I found that a 'reasonable' number helps to make 
#initial data visualisations easier to read)
top_venues_lim = 5

In [ ]:
#sort and get counts of venue categories by district/party
df_sort = pd.DataFrame(data=None)

df_sort = district_venues[['District', 
                           'Party', 
                           'Venue Category']].copy()

df_sort = df_sort.sort_values(by=(['District']),
                                ascending=True).reset_index()

df_sort = df_sort.groupby(['District', 'Party', 'Venue Category'],
                                  as_index=False).count()
df_sort = df_sort.sort_values(by=(['District', 'index']),
                             ascending=False)

df_sort = df_sort.reset_index()

districts = []
districts = df_sort['District'].unique()

df_tops = pd.DataFrame(data = None)

#quick and dirty loop to get the top 'n' venues data for each district
for distr in districts:
    df_temp0 = pd.DataFrame(data=None)
    df_temp0 = df_temp0.append(df_sort.loc[df_sort['District']==distr])
    df_temp1 = df_temp0.head(top_venues_lim)
    df_tops = df_tops.append(df_temp1)

df_tops=df_tops.rename({'index':'Venue Count'}, axis=1)

df_tops.head(3)

Plot the venue data in a stacked bar chart to illustrate preferences by district.

In [ ]:
#set color range depending on how many venue categories are in the df
#(this technique was applied in the Course 9 kmeans lab).

unique_categories = len(df_tops['Venue Category'].unique())
x = np.arange(unique_categories)

ys = [i + x + (i*x)**2 for i in range(unique_categories)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

df_cats=pd.DataFrame(data=None)
df_cats['cat'] = df_tops['Venue Category'].unique()
df_cats = df_cats.sort_values(by=['cat'], 
                              ascending = True).reset_index()

#now add rainbow colours to the categories
df_cats['colour'] = rainbow

#df_cats.head(3)

In [ ]:
#set up bar chart object by looping through district venue data, 
#plotting each district's bar for that category using a colour from 
#the colour mapping we've just set up

#set these variables to help stack the bars properly, then loop
#to add each venue category total onto the district bar
prevbot=0
saved=""
for distr, category, count in zip(df_tops['District'], 
                                    df_tops['Venue Category'],
                                    df_tops['Venue Count']):
#    print ('i: ', i, ' ', distr, ' ', category, ' ', count)
    catcol = df_cats['colour'].loc[df_cats['cat']==category]
    if distr != saved:
        plt.bar(distr, count, width = 0.5, bottom = 0, color=catcol)
        prevbot = count
        saved = distr
    else:
        plt.bar(distr, count, width = 0.5, bottom = prevbot,
                color=catcol)
        prevbot = prevbot + count
    
plt.title(
    "Venues in Common across Top {} Categories by District".format(
        top_venues_lim),
             fontsize=12)

plt.xticks(df_tops['District'].unique(), 
                    fontsize=10)

plt.xlabel("Districts", fontsize=10)
plt.ylabel("Venue Count", fontsize=10)


#due to the stacking, the legend had many duplicates; to fix this, I 
#had to reset the legend handles and colour.  
#Also had to shift the legend outside the plot area (via 'bbox_') as 
#it is quite long despite getting rid of duplicates
plt.legend(df_cats['cat'], fontsize=10, bbox_to_anchor=(1, 1), 
           bbox_transform=plt.gcf().transFigure)

legnd=plt.gca().get_legend()

for i in range(unique_categories):
    legnd.legendHandles[i].set_color(df_cats['colour'][i])

plt.figure(figsize=(16,10))

plt.show()

This chart reflects numbers of venues of the top venue types per district, as returned from FourSquare (limited to 1000 entries per district within 3200 meters, approx. 2 miles, of the district head office).  The data suggests that there are certain venue preferences in common across a number of districts, for example, Fast Food restaurants, American restaurants and Sandwich places).  These venues may be of particular use during capaign advertisement placement selection.   

***********************************************************************

<h3 align=left><font size = 3>Step 3 - Iterate: repeat Data Sourcing,  Visualisation and Analysis for Counties:</font></h3>

***Gather data by county that could inform other means of grouping voters than the currently-gerrymandered district borders.  This 'new' view of voters can help direct the focus of campaign advisors and consultants.***

Obtain county information on voter population size, age ranges and income levels in each county to use as features in clustering later _(note: unfortunately I could not source equivalent information on 'education')._

In [ ]:
#pull per-capita personal income (line 110), 
#except 'GeoName' = "Georgia"
csv_path="/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/Coursera_Capstone/CAINC30_GA_1969_2018.csv"
df_temp=pd.read_csv(csv_path)

df_temp = df_temp.loc[ (
    (df_temp['GeoName'] != "Georgia") & (df_temp['LineCode'] == 110) ) ]


In [ ]:
#prep counties income data
df_counties_merged = pd.DataFrame(data=None)

df_counties_merged = df_temp[['GeoName', '2018']].copy()

cols=['County', '2018 Per Capita Income (000s)'] 

df_counties_merged.columns = cols

df_counties_merged['2018 Per Capita Income (000s)'] = \
  (df_counties_merged['2018 Per Capita Income (000s)'] / 1000).round(2)


#make County upper case for later merging with other upper case data
df_counties_merged['County'] = \
    df_counties_merged['County'].apply(lambda x: x.upper())

df_counties_merged.head(3)

In [ ]:
#Load the longitudes and latitudes for the various counties
df_counties_merged['Latitude'] = None
df_counties_merged['Longitude'] = None

for i, county in enumerate(df_counties_merged['County']):
    g = geocoder.arcgis(county)
#    print('county: ', county)
    df_counties_merged.iloc[ 
      i, df_counties_merged.columns.get_loc('Latitude')] = g.latlng[0]
    df_counties_merged.iloc[
      i, df_counties_merged.columns.get_loc('Longitude')] = g.latlng[1]
    
#...and then drop the Georgia suffix from county names
df_counties_merged['County'] = \
    df_counties_merged['County'].str.replace(", GA","")

df_counties_merged.head(3)

In [ ]:
#pull 2018 demographic data by county from constructed webpage 
#(courtesy of US Census Bureau)
url = \
    'http://sos.ga.gov/admin/files/Active_Voters_by_Race_Gender_and_Age_as_of_May_1_2020.xlsx'
outfile = '/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/Coursera_Capstone/Active_Voters_by_Race_Gender_and_Age_as_of_May_1_2020.xlsx'

wget.download(url, outfile)
df_county_demogs = pd.read_excel(outfile, skiprows=range(8))

#get rid of county totals rows (these have NaN in certain columns)
df_county_demogs=df_county_demogs.dropna(axis=0)


In [ ]:
#keep slicing to minimum (for clarity) but shorten excel label names
cols=['COUNTY ID', 
      'COUNTY NAME', 
      'VOTER AGE RANGE',
      'AI MALE', 'AI FEM', 'AI UNK', 
      'AP MALE', 'AP FEM', 'AP UNK',
      'BH MALE', 'BH FEM', 'BH UNK', 
      'HP MALE', 'HP FEM', 'HP UNK',
      'WH MALE', 'WH FEM', 'WH UNK', 
      'OT MALE', 'OT FEM', 'OT UNK',
      'UK MALE', 'UK FEM', 'UK UNK',
        'TOTAL VOTERS']
df_county_demogs.columns = cols

df_county_demogs.head(3)

In [ ]:
#prep age data, incl pivoting to make age range columns instead of rows

df_age_temp = pd.DataFrame(data=None) 

df_age_temp = df_county_demogs[['COUNTY NAME',
                            'VOTER AGE RANGE',
                            'TOTAL VOTERS']]

#prep age data, incl pivoting to make age range columns instead of rows
df_age_temp = df_age_temp.pivot(index='COUNTY NAME', columns = 
    'VOTER AGE RANGE', 
     values = 'TOTAL VOTERS').reset_index().rename_axis(None, axis=1)

df_age_temp.head(3)
 

In [ ]:
#consolidate age ranges - follow the pattern used for the district
#demographic analysis earlier, incl adding % for later use
#

df_age_temp['18-29 years old (000s)'] = (df_age_temp['18-24'] + \
                    df_age_temp['25-29']) / 1000
 
df_age_temp['30-44 years old (000s)'] = (df_age_temp['30-34'] + \
                    df_age_temp['35-39'] + df_age_temp['40-44']) / 1000

df_age_temp['45-64 years old (000s)'] = (df_age_temp['45-49'] + \
                    df_age_temp['50-54'] +\
                    df_age_temp['55-59'] + df_age_temp['60-64']) / 1000

df_age_temp['65 years old and older (000s)']=df_age_temp['65+'] / 1000


#drop unncesssary columns
df_age_temp.drop(['17 1/2', '18-24', '25-29',
                 '30-34', '35-39', '40-44',
                 '45-49', '50-54', '55-59',
                 '60-64', '65+'], axis=1, inplace=True)

df_age_temp.head(3)

In [ ]:
#prep ethnicity demographics incl column totals to use for clustering
#(will consolidate certain data and not use the more granular columns).

#below adds columns together to consolidate ethnicity detail, then
#drops the unnec columns
df_county_demogs['Hispanic (000s)'] = \
    (df_county_demogs['HP MALE'] + \
    df_county_demogs['HP FEM'] + \
    df_county_demogs['HP UNK']) / 1000

df_county_demogs['White (000s)'] = \
    (df_county_demogs['WH MALE'] + \
    df_county_demogs['WH FEM'] + \
    df_county_demogs['WH UNK']) / 1000

df_county_demogs['Black (000s)'] = \
    (df_county_demogs['BH MALE'] + \
    df_county_demogs['BH FEM'] + \
    df_county_demogs['BH UNK']) / 1000

df_county_demogs['Asian or Pacific Isl (000s)'] = \
    (df_county_demogs['AP MALE'] + \
    df_county_demogs['AP FEM'] + \
    df_county_demogs['AP UNK']) / 1000

df_county_demogs['Other Ethn (000s)'] = \
    (df_county_demogs['AI MALE'] + \
    df_county_demogs['AI FEM'] + \
    df_county_demogs['AI UNK'] + \
    df_county_demogs['OT MALE'] + \
    df_county_demogs['OT FEM'] + \
    df_county_demogs['OT UNK'] + \
    df_county_demogs['UK MALE'] + \
    df_county_demogs['UK FEM'] + \
    df_county_demogs['UK UNK']) / 1000

df_county_demogs.drop([
      'AI MALE', 'AI FEM', 'AI UNK', 
      'AP MALE', 'AP FEM', 'AP UNK',
      'BH MALE', 'BH FEM', 'BH UNK', 
      'HP MALE', 'HP FEM', 'HP UNK',
      'WH MALE', 'WH FEM', 'WH UNK', 
      'OT MALE', 'OT FEM', 'OT UNK',
      'UK MALE', 'UK FEM', 'UK UNK'
        ], axis=1, inplace=True)

#now group the demographics numbers by county
df_county_demogs = df_county_demogs.sort_values(by=
                        ['COUNTY ID'], ascending=True)
df_county_demogs = df_county_demogs.groupby(['COUNTY ID', 
                                            'COUNTY NAME'],
                                           as_index = False).sum()

df_county_demogs.head(3)

In [ ]:
#make the TOTAL VOTERS column 000s and rename cols (for consistency)
df_county_demogs['TOTAL VOTERS'] = \
    df_county_demogs['TOTAL VOTERS'] /1000

df_county_demogs.rename({'TOTAL VOTERS':'Voting Pop (000s)',
                        'COUNTY ID':'County Id'}, axis=1,
                        inplace=True)


In [ ]:
#merge ethnicity county demographics data
df_counties_merged = pd.merge(left=df_counties_merged, 
                              right=df_county_demogs, 
                        left_on='County', right_on='COUNTY NAME')

df_counties_merged.drop(['COUNTY NAME'], 
                        axis=1, inplace=True)

df_counties_merged.head(3)

In [ ]:
#and merge age demographics data
df_counties_merged = pd.merge(left=df_counties_merged, 
                              right=df_age_temp, 
                        left_on='County', right_on='COUNTY NAME')

df_counties_merged.drop(['COUNTY NAME'], 
                        axis=1, inplace=True)

df_counties_merged.head(3)

In [ ]:
#Lastly, add demographics percentages
df_counties_merged['Hispanic %'] = \
    ((df_counties_merged['Hispanic (000s)'] / \
      df_counties_merged['Voting Pop (000s)']) *100).round(2)
df_counties_merged['White %'] = \
    ((df_counties_merged['White (000s)'] / \
      df_counties_merged['Voting Pop (000s)']) *100).round(2)
df_counties_merged['Black %'] = \
    ((df_counties_merged['Black (000s)'] / \
      df_counties_merged['Voting Pop (000s)']) *100).round(2)
df_counties_merged['Asian or Pacific Isl %'] = \
    ((df_counties_merged['Asian or Pacific Isl (000s)'] / \
      df_counties_merged['Voting Pop (000s)']) *100).round(2)
df_counties_merged['Other Ethn %'] = \
    ((df_counties_merged['Other Ethn (000s)'] / \
      df_counties_merged['Voting Pop (000s)']) *100).round(2)

df_counties_merged['18-29 years old %'] = \
    ((df_counties_merged['18-29 years old (000s)'] / 
        df_counties_merged['Voting Pop (000s)']) *100).round(2)
df_counties_merged['30-44 years old %'] = \
    ((df_counties_merged['30-44 years old (000s)'] / \
      df_counties_merged['Voting Pop (000s)']) \
         *100).round(2)
df_counties_merged['45-64 years old %'] = \
    ((df_counties_merged['45-64 years old (000s)'] / \
      df_counties_merged['Voting Pop (000s)']) \
         *100).round(2)
df_counties_merged['65 years old and older %'] = \
    ((df_counties_merged['65 years old and older (000s)'] /  
      df_counties_merged['Voting Pop (000s)']) *100).round(2)

df_counties_merged.head(3)

***********************************************************************

Now see what clusters are available given the county demographic features gathered above; this could yield further campaign-targetting intelligence and inform debates on the practice of gerrymandering.

In [ ]:
#select interesting columns to use for our kmeans clustering
kcols=['2018 Per Capita Income (000s)', 
       'Voting Pop (000s)', 'Hispanic %', 'White %', 'Black %', 
       'Asian or Pacific Isl %', 'Other Ethn %', '18-29 years old %', 
       '30-44 years old %', '45-64 years old %', 
       '65 years old and older %']

In [ ]:
#we'll set up a separate copy of the merged county data (makes it 
#easier to manipulate dataframes without unintended data consequences!)
df_county_clusters = pd.DataFrame(data=None)
df_county_clusters = df_counties_merged[kcols].copy()

#we'll drop the categorical value
#df_county_clusters = df_county_clusters.drop('County', axis=1)

#...and we'll normalise the data (as we're using a mathematical model)
X=df_county_clusters.values[:,1:]
#this line is a precautionary move, in case I missed a NaN
X=np.nan_to_num(X)

#execute normaliser
df_county_clusters = StandardScaler().fit_transform(X)


In [ ]:
#In order to use the best 'kclusters', we'll run the kmeans algo a 
#few times, plotting the average mean distance between each datapoint
#and its respective centroid...then we'll see which cluster produced
#the most dramatic difference and will use that (this will require
#computing the "Sum of Squared differences", referred to as 'SSE' in
#the IBM course)
SSE=[]

#we'll test 14 kmeans cluster numbers (somewhat randomly selected...I 
#just went with the number of existing districts)
krange = np.arange(1,14)


In [ ]:
#execute for loop of kmeans algos, 
for knum in krange:
    ktest = KMeans(n_clusters=knum, \
                random_state=0).fit(df_county_clusters)
    SSE.append(ktest.inertia_)
    

In [ ]:
#now plot the average distance values and find the "elbow"
plt.plot(krange, SSE, 'bx-')
plt.xlabel('cluster')
plt.ylabel('Sum of squared distances')
plt.title('Clusters (k) for Georgia counties kmeans')
plt.show()

In [ ]:
#The test yielded an unsatisfactory result, in that the most dramatic
#change in difference appears to be at 2, a very limited cluster 
#number, and next most dramatic seems to be 10 and 12, which are too
#many clusters for my purposes.  I will choose the next nearest change 
#(as far as I am able to tell by visual inspection), which is 5.

kclusters = 5

#...then add the cluster label into list of kcols columns of interest
kcols.append('Cluster')


In [ ]:
#execute the kmeans clustering for real this time
kmeans = KMeans(n_clusters=kclusters, \
                random_state=0).fit(df_county_clusters)

klabels = kmeans.labels_


In [ ]:
#we'll add column to reflect the cluster to which a county was
#assigned (from klabels, created after k-means executed)
df_counties_merged['Cluster'] = None

df_counties_merged['Cluster'] = klabels

#check the cluster labels
df_counties_merged['Cluster'].unique()

Now plot certain cluster statistics (county counts, population sizes, average income, ethnicity, age ranges) from each cluster to assess defining features.  

(Later, we will also plot these clusters on a Georgia map to aid our assessment).

In [ ]:
#now get the count and voter population and income average in columns
df_cluster_stats = pd.DataFrame(data=None)

for i in range(0, kclusters):
    temp=[]

    temp=df_counties_merged[kcols].loc[
            df_counties_merged['Cluster']==i].groupby(
                df_counties_merged['Cluster']).count()
    
    temp['Total Voters (000s)']= \
        df_counties_merged['Voting Pop (000s)'].loc[
            df_counties_merged['Cluster']==i].groupby(
                df_counties_merged['Cluster']).sum()
    
    temp['2018 Per Capita Income (000s)']= \
        df_counties_merged['2018 Per Capita Income (000s)'].loc[
            df_counties_merged['Cluster']==i].groupby(
                df_counties_merged['Cluster']).mean()
    
    
    df_cluster_stats=df_cluster_stats.append(temp)


In [ ]:
#plot these stats and income next to each other using 'subplot' (not taught
#in the course, but quite handy and easy to learn from matplotlib doc)

fig, ax=plt.subplots(ncols=3, figsize=(16,6))

#plot a bar chart for each cluster's stat result
ax1=df_cluster_stats['Cluster'].plot(kind='bar', 
    title='Number of Counties',
    fontsize=12, rot=0, color='cadetblue', ax=ax[0])
#add totals at top of bar for better visualisation
for i in ax1.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax1.annotate('{:}'.format(height), (x + 0.1, y + height + 1.0), fontsize=10, 
                 color='black')
    
#...and plot the voter totals
ax2=df_cluster_stats['Total Voters (000s)'].plot(kind='bar', 
    title='Total Voters (000s)',
    fontsize=12, rot=0, color='darksalmon', ax=ax[1])
#add totals at top of bar for better visualisation
for i in ax2.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax2.annotate('{0:.3f}'.format(height), (x - 0.2, y + height + 1.0), fontsize=10, 
                 color='black')

#...and plot the voter totals
ax3=df_cluster_stats['2018 Per Capita Income (000s)'].plot(kind='bar', 
    title='Average per capita Income (000s)',
    fontsize=12, rot=0, color='palevioletred', ax=ax[2])
#add totals at top of bar for better visualisation
for i in ax3.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax3.annotate('{0:.3f}'.format(height), (x - 0.1, y + height + 1.0), fontsize=10, 
                 color='black')

plt.show()

We can see that the clusters vary in terms of number of counties and voter population size, and are mostly on par regarding average income with one exception.

Cluster 3 stands out as having the largest voter population size whilst having the smallest number of counties. It also stands out as having the highest average per-capita income at 59.3k, with the other clusters roughly on par with one another (between 33.3k to 38.1k average pe capita income).

Cluster 1 also stands out as having the smallest voter population size and second smallest number of constituent counties.

In [ ]:
#prep for remaining cluster bar charts.  Set colours and  
#get rid of the large value voting population (skews bar charts) and
#superfluous cluster col (cluster is the index for the plot df)
kcols.remove("Voting Pop (000s)")
kcols.remove("2018 Per Capita Income (000s)")
kcols.remove("Cluster")


In [ ]:
#now get averages for each cluster...

df_cluster_stats = pd.DataFrame(data=None)

for i in range(0, kclusters):
    temp=[]
    temp=df_counties_merged[kcols].loc[
                df_counties_merged['Cluster']==i].groupby(
                        df_counties_merged['Cluster']).mean()
    df_cluster_stats=df_cluster_stats.append([temp])


In [ ]:
#...and now plot the averages; plot a bar chart for each cluster's stat result
fig, ax=plt.subplots(ncols=2, figsize=(16,12))

#plot a bar chart for each cluster's stat result
ax1=df_cluster_stats[['Hispanic %', 'White %', 'Black %', 
       'Asian or Pacific Isl %', 'Other Ethn %']].plot(kind='bar', 
    title="Average of Counties' Ethnicity Proportions",
    fontsize=12, rot=0, ax=ax[0], width=0.75)
#add totals at top of bar for better visualisation
for i in ax1.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax1.annotate('{:.1f}'.format(height), (x - 0.02, y + height + 1.0), fontsize=10, 
                 color='black')
ytcks=np.arange(0,85,10)
ax1.set_yticklabels("")
ax1.set_ylabel("%")
ax1.set_yticks(ytcks)
    
cols2=['18-29 years old %', '30-44 years old %', '45-64 years old %', 
       '65 years old and older %']
ax2=df_cluster_stats[cols2].plot(kind='bar', 
    title="Average of County Voter Age Range Proportions",
    fontsize=12, rot=0, ax=ax[1], width=0.75)
#add totals at top of bar for better visualisation
for i in ax2.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax2.annotate('{:.1f}'.format(height), (x - 0.02, y + height + 0.8), fontsize=10, 
                 color='black')    
ytcks2=np.arange(0,65,10)
ax2.set_yticklabels("")
ax2.set_ylabel("%")
ax2.set_yticks(ytcks2)

plt.figure(figsize=(16,12))

plt.show()

For ethnicity and age demographics, we can see Clusters 1 and 2 have the highest proportion of white ethnicity amongst voters, with Cluster 1 also having the largest percentage of older (65 years and above) voters. 

Cluster 3 appears most mixed both in terms of ethnicity and age ranges, followed by Cluster 4 with a more evenly distributed age range than Cluster 3; both clusters have a lower proportion of older voters as compared to other clusters.


***********************************************************************

Now let's see what the clusters look like on the map relative to the position of the Districts (as indicated by District Office, as I was unable to source a usable Georgia congressional district map)

In [ ]:
#set up a base layer of state of Georgia (this was the only usable map 
#I could find)
georgia = geocoder.osm('Georgia, USA')
georgia.latlng

georgia_map=folium.Map(location=(georgia.latlng), 
                       zoom_start=7)

georgia_county_data = \
  '/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/Coursera_Capstone/cb_2018_13_sldu_500k.geojson'


#use choropleth to easily highlight the state of Georgia in the US map
georgia_map.choropleth(geo_data = georgia_county_data,
                            key_on='feature.properties.name',
                            fill_color='YlOrRd', 
                            fill_opacity=0.05, 
                            line_opacity=0.3)


#add each district office locations used in this analysis in below loop
for lat, lon, poi, pcolor in zip(df_districts_merged['Latitude'], 
                        df_districts_merged['Longitude'], 
                        df_districts_merged['District - Party'],
                        df_districts_merged['party_color']): 

    label = folium.Popup(poi, 
                         max_width=50,
                         parse_html=True)
    
#Folium/leaflet popup color pallette is more limited than matplotlib so
#need to change so district party colour is visible in map
    if pcolor=='indianred':
        colour='red'
    else:
        colour='blue'

    folium.Marker(
        [lat, lon],
        popup=label,
        icon=folium.Icon(icon='info-sign', color=colour)
                ).add_to(georgia_map)


fig = Figure(width=500, height=600)
fig.add_child(georgia_map)

georgia_map

'Blue' popups represent Democratic electorate offices, and 'Red' popups represent Republican electoral offices.

Given the redistricting requirement of equal approximate population size, it is unsurprising to see a cluster of district offices around the Atlanta region (an area of approximately 469,000 people in 2015, the year of the last redistricting). (source: wikipedia)

In [ ]:
#...and lastly, see where (approximately) our new clusters are located.
#To avoid cluttered map, get rid of the district map pops (by reloading
#the georgia map)
georgia_map=folium.Map(location=(georgia.latlng), 
                       zoom_start=7)
georgia_map.choropleth(geo_data = georgia_county_data,
                            key_on='feature.properties.name',
                            fill_color='YlOrRd', 
                            fill_opacity=0.05, 
                            line_opacity=0.3)

#...and then set colours for clusters (use cluster number to index 
#into colours list) and plot the counties
cluster_colours=['blue', 'green', 
           'red', 'cyan', 'magenta']
#                'yellow', 'gray']
df_clusterc=pd.DataFrame(data=None, columns=['cluster', 'colour'])
df_clusterc['cluster'] = df_counties_merged['Cluster'].unique()
df_clusterc['colour'] = cluster_colours
df_clusterc = df_clusterc.sort_values(by='cluster', ascending=True)


for lat, lng, county, cluster in zip(df_counties_merged['Latitude'], 
                           df_counties_merged['Longitude'], 
                           df_counties_merged['County'],
                            df_counties_merged['Cluster']):
    
    clcolour = df_clusterc['colour'].loc[(df_clusterc['cluster']== \
                                       cluster)].tolist()
#    print('cluster: ', cluster, ' colour: ', clcolour)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        weight=1,
        color='yellow',
        fill=True,
        popup=county,
        fill_color=clcolour,
        fill_opacity=0.6
    ).add_to(georgia_map)


fig = Figure(width=500, height=600)
fig.add_child(georgia_map)

georgia_map

In [ ]:
#as there is no easy way to visualise a legend in Folium, will create
#a quick df with the colours used

df_clusterc.style.hide_index()

Counties of the same clusters are, for the most part, geographically dispersed, however these clusters may still suggest a prioritisation or campaign alignment strategy that extends beyond normal 'district-centric' targetting.  Given the presumed gerrymandering applied to the last redistricting in Georgia, this may be important to state-wide election campaigns.


***********************************************************************

<h3 align=left><font size = 3>Step 4 - Results Discussion:</font></h3>

***Below consolidates the various observations on districts and counties that may be of most interest to campaign advisors and consultants.***

To be written into report

***********************************************************************

<h3 align=left><font size = 3>Step 5 - Conclusions:</font></h3>

***Below provides conclusions related to the central question "What can be done to counter the effects of gerrymandering?"***

To be written into report